In [38]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_train = pd.read_csv('../data/Train_data.csv')
raw_test = pd.read_csv('../data/Test_data.csv')

In [3]:
print(raw_train.info())
raw_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167408 entries, 0 to 167407
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   S1      167408 non-null  int64
 1   C1      167408 non-null  int64
 2   S2      167408 non-null  int64
 3   C2      167408 non-null  int64
 4   S3      167408 non-null  int64
 5   C3      167408 non-null  int64
 6   S4      167408 non-null  int64
 7   C4      167408 non-null  int64
 8   S5      167408 non-null  int64
 9   C5      167408 non-null  int64
 10  Class   167408 non-null  int64
dtypes: int64(11)
memory usage: 14.0 MB
None


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,Class
0,4,10,4,12,4,11,4,5,4,4,5
1,1,7,3,6,4,1,2,2,1,12,0
2,4,8,1,8,3,1,1,1,2,11,2
3,1,6,2,4,3,4,2,5,2,6,2
4,3,11,2,11,1,12,3,4,2,12,2


In [4]:
print(raw_test.shape)
raw_test.head(3)

(55803, 10)


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,2,2,3,12,3,8,1,12,3,9
1,1,9,4,5,2,6,4,11,4,8
2,1,1,4,12,1,12,3,3,3,1


Attribute Information:

   1) S1 - “Suit of card #1”

      Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

 

   2) C1 - “Rank of card #1”

      Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

 

   3) S2 - “Suit of card #2”

      Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

 

   4) C2 - “Rank of card #2”

      Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

 

   5) S3 - “Suit of card #3”

      Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

 

   6) C3 - “Rank of card #3”

      Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

 

   7) S4 - “Suit of card #4”

      Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

 

   8) C4 - “Rank of card #4”

      Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

 

   9) S5 - “Suit of card #5”

      Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

 

   10) C5 - “Rank of card 5”

      Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

 

   11) Class - “Poker Hand”

      Ordinal (0-9)

0: Nothing in hand; not a recognized poker hand 

1: One pair; one pair of equal ranks within five cards

2: Two pairs; two pairs of equal ranks within five cards

3: Three of a kind; three equal ranks within five cards

4: Straight; five cards, sequentially ranked with no gaps ---------done

5: Flush; five cards with the same suit ------------done

6: Full house; pair + different rank three of a kind ------------done

7: Four of a kind; four equal ranks within five cards ------ done

8: Straight flush; straight + flush  -------------done

9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush ------ done

In [29]:
s = raw_train.sample(1)
s

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,Class
81106,2,9,3,10,2,12,4,8,1,6,0


In [126]:
def is_straight(vals):
    s_vals = sorted(vals)
    
    if len(set(vals)) != 5:
        return False
    
    diff = [s_vals[i] - s_vals[i-1] for i in range(1, 5)]
    
    if len(set(diff)) == 1 and diff[0] == 1:
        return True
    elif s_vals == [1, 10, 11, 12, 13]:
        return True
                    
    return False

In [127]:
# is_straight([10, 12, 11, 13, 1])

In [128]:
def get_labels(row):
    vals = row.values.tolist()[:10]
#     print(vals)
    
    suits = vals[::2]
    suit_counter = Counter(suits)
    
    cards = vals[1::2]
    card_counter = Counter(cards)

    # royal flush condition
    if len(suit_counter) == 1 and sorted(cards) == [1, 10, 11, 12, 13]:
            return 9
        
    # straight + flush
    elif is_straight(cards) and len(suit_counter) == 1:
        return 8
    
    # four of a kind
    elif sorted(list(card_counter.values())) == [1, 4]:
        return 7
    
    # full house
    elif sorted(list(card_counter.values())) == [2, 3]:
        return 6
        
    # flush
    elif len(suit_counter) == 1:
        return 5    
    
    # straight
    elif is_straight(cards):
        return 4
 
    # three of a kind
    elif sorted(list(card_counter.values())) == [1, 1, 3]:
        return 3
    
    # two pairs
    elif sorted(list(card_counter.values())) == [1, 2, 2]:
        return 2
    
    # one pair
    elif sorted(list(card_counter.values())) == [1, 1, 1, 2]:
        return 1
    
    # nothing
    else:
        return 0

In [129]:
preds = []

for i, row in tqdm(raw_train.iterrows(), total=raw_train.shape[0]):
    ans = get_labels(row)
    preds.append(ans)

  0%|          | 0/167408 [00:00<?, ?it/s]

In [130]:
train = raw_train.copy()
train['preds'] = preds

In [131]:
mask = train['Class'] != train['preds']
wrongs = train.loc[mask]
wrongs

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,Class,preds


In [122]:
test_preds = []

for i, row in tqdm(raw_test.iterrows(), total=raw_test.shape[0]):
    ans = get_labels(row)
    test_preds.append(ans)

  0%|          | 0/55803 [00:00<?, ?it/s]

In [124]:
sub = pd.DataFrame({'prediction': test_preds})
print(sub.shape)
sub.head()

(55803, 1)


,prediction
0,1
1,0
2,2
3,1
4,5


In [125]:
sub.to_csv('../submissions/sub1.csv', index=False)